# Compression – Load Data and compress vectors

## Get keys and urls

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_URL = os.getenv("OPENAI_URL")

print(f"Weaviate Key:{WEAVIATE_KEY}")
print(f"OpenAI API Key: {OPENAI_API_KEY[:20]}")
print(f"OpenAI URL: {OPENAI_URL}")

## Connect to Weaviate

In [ ]:
import weaviate
from weaviate.classes.init import Auth

# Connect to the local instance
client = weaviate.connect_to_local(
  host="127.0.0.1", # the address to the learner's instance
  port=8080,
  grpc_port=50051,
  auth_credentials=Auth.api_key(WEAVIATE_KEY),
  headers={
    "X-OpenAI-Api-Key": OPENAI_API_KEY
  }
)

print(client.is_ready())

## Create Collection with SQ configuration

[Docs: Rotational Quantization (RQ)](https://weaviate.io/developers/weaviate/configuration/compression/rq-compression)

> Note: Rotational Quantization (RQ) does not require a training phase.<br/>
> RQ begins compressing vectors immediately upon insertion, without waiting for a minimum number of objects or a training step.
>
> This makes RQ ideal for applications that need immediate compression and fast setup, as vectors are compressed and searchable as soon as they are added to the collection.

In [ ]:
from weaviate.classes.config import Configure

client.collections.delete("WikiQ")

client.collections.create(
    name="WikiQ",

    vector_config=[
        Configure.Vectors.text2vec_openai(
            name="main_vector",
            model="text-embedding-3-small",
            base_url=OPENAI_URL,
            source_properties=['title', 'text'],

            # Configure RQ
            vector_index_config=Configure.VectorIndex.hnsw(
                quantizer=Configure.VectorIndex.Quantizer.rq(
                    rescore_limit=200,  # Number of overfetched candidates used for rescoring
                    bits=8              # Number of bits (only 8 is supported)
                )
            ),
        )
    ],
)

## The rest is the same

In [ ]:
from data_loader import import_wiki_data
import_wiki_data(client, "WikiQ", 25000)

In [ ]:
WikiQ = client.collections.get("WikiQ")
WikiQ.aggregate.over_all()

## Clean up

In [ ]:
# client.collections.delete("WikiQ")

## Close the client

In [ ]:
client.close()